In [17]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

1.14.0


In [2]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

W1005 13:27:18.200756 14244 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [60]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [61]:
tokenizer = Tokenizer(1e4, oov_token = '<OOV>')
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=120, truncating='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=120)

In [30]:
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [33]:
print(decode_review(padded[1]))
print(testing_sentences[1])

in its many forms kids may be the target audience but it 's fun for everyone to laugh at its comical silliness yet at the same time root for the good guys to <OOV> and save the world the acting is cheesy in places but that is the charm there are several lines of corny dialog possibly translation errors or possibly intentional jokes by the movie makers and you 'll find yourself <OOV> these absurd lines later br br admittedly this film is not high in production quality or budget however for what it is campy sci fi it 's enjoyable for some laughs i recommend it to anyone with a sense of humor for that sort of thing '
b'If you love Japanese monster movies, you\'ll love this action packed battle pitting an alien invader, intent upon conquering the Earth, and a "Giant Robot" with an armory of super weaponry. The alien, "Emporer Guillotine," from the planet Gargoyle, has a army of thugs called, (of course) "the gargoyle gang," as well as an endless supply of immense hostile creatures that are

In [59]:
tf.reset_default_graph()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=120),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [93]:
history = model.fit(padded, training_labels_final, epochs=10, 
          validation_data=(testing_padded, testing_labels_final),
         verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 - 3s - loss: 9.5898e-05 - acc: 1.0000 - val_loss: 0.8295 - val_acc: 0.8275
Epoch 2/10
25000/25000 - 3s - loss: 5.9991e-05 - acc: 1.0000 - val_loss: 0.8625 - val_acc: 0.8275
Epoch 3/10
25000/25000 - 3s - loss: 3.7705e-05 - acc: 1.0000 - val_loss: 0.8951 - val_acc: 0.8274
Epoch 4/10
25000/25000 - 2s - loss: 2.3444e-05 - acc: 1.0000 - val_loss: 0.9280 - val_acc: 0.8278
Epoch 5/10
25000/25000 - 2s - loss: 1.5192e-05 - acc: 1.0000 - val_loss: 0.9570 - val_acc: 0.8271
Epoch 6/10
25000/25000 - 2s - loss: 9.3921e-06 - acc: 1.0000 - val_loss: 0.9882 - val_acc: 0.8274
Epoch 7/10
25000/25000 - 2s - loss: 5.9938e-06 - acc: 1.0000 - val_loss: 1.0156 - val_acc: 0.8273
Epoch 8/10
25000/25000 - 2s - loss: 3.8241e-06 - acc: 1.0000 - val_loss: 1.0426 - val_acc: 0.8282
Epoch 9/10
25000/25000 - 2s - loss: 2.4383e-06 - acc: 1.0000 - val_loss: 1.0685 - val_acc: 0.8276
Epoch 10/10
25000/25000 - 2s - loss: 1.5302e-06 - acc: 1.0000 - val_

In [94]:
e = model.layers[0]

In [99]:
weights = e.get_weights()[0]

In [100]:
weights.shape

(10000, 16)

In [102]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, 10000):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word+'\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()